In [7]:
import hail as hl

In [8]:
hl

<module 'hail' from '/opt/conda/lib/python3.7/site-packages/hail/__init__.py'>

In [8]:
mt1 = hl.read_matrix_table('gs://cpg-tob-wgs-test/mt/v7.mt/')

In [9]:
# 175,993,904 variants, subset of 24 donors
mt1.count()

(175993904, 24)

In [10]:
biallelic_mt = mt1.filter_rows(hl.len(mt1.alleles) == 2)

In [11]:
# 17,988,289 variants are bi-allelic
biallelic_mt.count()

(17988289, 24)

In [ ]:
# gs://cpg-tob-wgs-main/tob_wgs_vep/v1/vep105_GRCh38.mt/

In [2]:
# mt = hl.read_matrix_table('gs://cpg-tob-wgs-test/tob_wgs_vep/v0/vep105_GRCh38.mt')
mt = hl.read_matrix_table('gs://cpg-tob-wgs-test/tob_wgs_vep/v1/vep105_GRCh38.mt')

Initializing Hail with default parameters...


2022-08-11 01:25:51 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-08-11 01:25:53 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.2.
  Compatibility is not guaranteed.


Running on Apache Spark version 3.1.2
SparkUI available at http://annas-notebook.australia-southeast1-a.c.notebooks-314505.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.73-c6f6f09cec08
LOGGING: writing to /home/jupyter/hail-20220811-0125-0.2.73-c6f6f09cec08.log


In [3]:
mt.count()

(77899783, 1061)

In [4]:
mt0 = mt.head(10000)
mt0.count()

(10000, 1061)

In [5]:
# genetic variants (defined by position (locus) and alleles (reference, alternative or viceversa?))
mt0.rows().select().show(5)

,
locus,alleles
locus<GRCh38>,array<str>
chr1:10011,"[""C"",""T""]"
chr1:10020,"[""A"",""C""]"
chr1:10027,"[""A"",""G""]"
chr1:10109,"[""AACCCT"",""A""]"
chr1:10114,"[""T"",""C""]"


In [7]:
# individual IDs (CPG ones, do not match the scRNA-seq ones)
mt0.s.show(5)

""
s
str
"""CPG18"""
"""CPG26"""
"""CPG34"""
"""CPG42"""
"""CPG59"""


In [6]:
mt0.describe()

----------------------------------------
Global fields:
    'bin_group_variant_counts': struct {
        non_lcr_bin: struct {
            snv: int32, 
            indel: int32
        }, 
        singleton_bin: struct {
            snv: int32, 
            indel: int32
        }, 
        biallelic_bin: struct {
            snv: int32, 
            indel: int32
        }, 
        biallelic_singleton_bin: struct {
            snv: int32, 
            indel: int32
        }, 
        raw_non_lcr_singleton_bin: struct {
            snv: int32, 
            indel: int32
        }, 
        raw_non_lcr_biallelic_bin: struct {
            snv: int32, 
            indel: int32
        }, 
        non_lcr_biallelic_singleton_bin: struct {
            snv: int32, 
            indel: int32
        }, 
        bin: struct {
            snv: int32, 
            indel: int32
        }, 
        raw_bin: struct {
            snv: int32, 
            indel: int32
        }, 
        raw_non_lcr_bia

In [32]:
mt0 = hl.variant_qc(mt0)

In [33]:
mt01 = mt0.filter_rows(mt0.variant_qc.AF[0] < 1)

In [ ]:
mt02 = mt01.filter_rows(mt01.vep.variant_class == 'SNV')

In [43]:
mt01.count()

(8909, 1061)

In [34]:
mt01.count_rows()

8909

In [40]:
# 40%
snp_maf_05 = mt01.aggregate_rows(hl.agg.count_where(mt01.variant_qc.AF[1] < 0.05))
print(f'Variant MAF < 0.05 = {snp_maf_05}')

Variant MAF < 0.05 = 3595


In [41]:
# 18%
snp_maf_01 = mt01.aggregate_rows(hl.agg.count_where(mt01.variant_qc.AF[1] < 0.01))
print(f'Variant MAF < 0.01 = {snp_maf_01}')

Variant MAF < 0.01 = 1602


In [47]:
# 12%
snp_maf_005 = mt01.aggregate_rows(hl.agg.count_where(mt01.variant_qc.AF[1] < 0.005))
print(f'Variant MAF < 0.005 = {snp_maf_005}')

Variant MAF < 0.005 = 1068


In [44]:
mt02 = mt01.filter_rows(mt01.vep.variant_class == 'SNV')

In [45]:
mt02.count_rows()

7184

In [46]:
# 35%
snp_maf_05 = mt02.aggregate_rows(hl.agg.count_where(mt02.variant_qc.AF[1] < 0.05))
print(f'Variant MAF < 0.05 = {snp_maf_05}')

Variant MAF < 0.05 = 2520


In [48]:
# 13%
snp_maf_01 = mt02.aggregate_rows(hl.agg.count_where(mt02.variant_qc.AF[1] < 0.01))
print(f'Variant MAF < 0.01 = {snp_maf_01}')

Variant MAF < 0.01 = 938


In [49]:
# 8%
snp_maf_005 = mt02.aggregate_rows(hl.agg.count_where(mt02.variant_qc.AF[1] < 0.005))
print(f'Variant MAF < 0.005 = {snp_maf_005}')

Variant MAF < 0.005 = 584


In [83]:
mt0.vep.select("assembly_name","allele_string","ancestral","most_severe_consequence","variant_class","start","end","strand","transcript_consequences").show(10)

+---------------+----------------+----------------------+----------------------+
| locus         | alleles        | <expr>.assembly_name | <expr>.allele_string |
+---------------+----------------+----------------------+----------------------+
| locus<GRCh38> | array<str>     | str                  | str                  |
+---------------+----------------+----------------------+----------------------+
| chr1:10011    | ["C","T"]      | "GRCh38"             | "C/T"                |
| chr1:10020    | ["A","C"]      | "GRCh38"             | "A/C"                |
| chr1:10027    | ["A","G"]      | "GRCh38"             | "A/G"                |
| chr1:10109    | ["AACCCT","A"] | "GRCh38"             | "ACCCT/-"            |
| chr1:10114    | ["T","C"]      | "GRCh38"             | "T/C"                |
| chr1:10114    | ["TA","T"]     | "GRCh38"             | "A/-"                |
| chr1:10119    | ["CT","C"]     | "GRCh38"             | "T/-"                |
| chr1:10120    | ["T","A"]      | "GRCh38"             | "T/A"                |
| chr1:10120    | ["T","C"]      | "GRCh38"             | "T/C"                |
| chr1:10120    | ["TA","T"]     | "GRCh38"             | "A/-"                |
+---------------+----------------+----------------------+----------------------+

+------------------+--------------------------------+----------------------+
| <expr>.ancestral | <expr>.most_severe_consequence | <expr>.variant_class |
+------------------+--------------------------------+----------------------+
| str              | str                            | str                  |
+------------------+--------------------------------+----------------------+
| NA               | "upstream_gene_variant"        | "SNV"                |
| NA               | "upstream_gene_variant"        | "SNV"                |
| NA               | "upstream_gene_variant"        | "SNV"                |
| NA               | "upstream_gene_variant"        | "deletion"           |
| NA               | "upstream_gene_variant"        | "SNV"                |
| NA               | "upstream_gene_variant"        | "deletion"           |
| NA               | "upstream_gene_variant"        | "deletion"           |
| NA               | "upstream_gene_variant"        | "SNV"                |
| NA               | "upstream_gene_variant"        | "SNV"                |
| NA               | "upstream_gene_variant"        | "deletion"           |
+------------------+--------------------------------+----------------------+

+--------------+---------------+
| <expr>.start | <expr>.strand |
+--------------+---------------+
|        int32 |         int32 |
+--------------+---------------+
|        10011 |             1 |
|        10020 |             1 |
|        10027 |             1 |
|        10110 |             1 |
|        10114 |             1 |
|        10115 |             1 |
|        10120 |             1 |
|        10120 |             1 |
|        10120 |             1 |
|        10121 |             1 |
+--------------+---------------+

+------------------------------------------------------------------------------+
| <expr>.transcript_consequences                                               |
+------------------------------------------------------------------------------+
| array<struct{allele_num: int32, amino_acids: str, appris: str, biotype: s... |
+------------------------------------------------------------------------------+
| [(1,NA,NA,"transcribed_unprocessed_pseudogene",1,NA,NA,NA,NA,NA,NA,NA,["u... |
| [(1,NA,NA,"transcribed_unprocessed_pseudogene",1,NA,NA,NA,NA,NA,NA,NA,["u... |
| [(1,NA,NA,"transcribed_unprocessed_pseudogene",1,NA,NA,NA,NA,NA,NA,NA,["u... |
| [(1,NA,NA,"transcribed_unprocessed_pseudogene",1,NA,NA,NA,NA,NA,NA,NA,["u... |
| [(1,NA,NA,"transcribed_unprocessed_pseudogene",1,NA,NA,NA,NA,NA,NA,NA,["u... |
| [(1,NA,NA,"transcribed_unprocessed_pseudogene",1,NA,NA,NA,NA,NA,NA,NA,["u... |
| [(1,NA,NA,"transcribed_unprocessed_pseudogene",1

In [81]:
snv_mt = mt0.filter_rows(mt0.vep.variant_class == "SNV")

In [84]:
snv_mt.vep.show(100)

+---------------+------------+-------------------+-------------------+
| locus         | alleles    | vep.assembly_name | vep.allele_string |
+---------------+------------+-------------------+-------------------+
| locus<GRCh38> | array<str> | str               | str               |
+---------------+------------+-------------------+-------------------+
| chr1:10011    | ["C","T"]  | "GRCh38"          | "C/T"             |
| chr1:10020    | ["A","C"]  | "GRCh38"          | "A/C"             |
| chr1:10027    | ["A","G"]  | "GRCh38"          | "A/G"             |
| chr1:10114    | ["T","C"]  | "GRCh38"          | "T/C"             |
| chr1:10120    | ["T","A"]  | "GRCh38"          | "T/A"             |
| chr1:10120    | ["T","C"]  | "GRCh38"          | "T/C"             |
| chr1:10126    | ["T","A"]  | "GRCh38"          | "T/A"             |
| chr1:10126    | ["T","C"]  | "GRCh38"          | "T/C"             |
| chr1:10132    | ["T","C"]  | "GRCh38"          | "T/C"             |
| chr1:10132    | ["T","G"]  | "GRCh38"          | "T/G"             |
| chr1:10137    | ["C","T"]  | "GRCh38"          | "C/T"             |
| chr1:10138    | ["T","A"]  | "GRCh38"          | "T/A"             |
| chr1:10138    | ["T","C"]  | "GRCh38"          | "T/C"             |
| chr1:10138    | ["T","G"]  | "GRCh38"          | "T/G"             |
| chr1:10139    | ["A","C"]  | "GRCh38"          | "A/C"             |
| chr1:10141    | ["C","G"]  | "GRCh38"          | "C/G"             |
| chr1:10144    | ["T","A"]  | "GRCh38"          | "T/A"             |
| chr1:10144    | ["T","C"]  | "GRCh38"          | "T/C"             |
| chr1:10144    | ["T","G"]  | "GRCh38"          | "T/G"             |
| chr1:10145    | ["A","C"]  | "GRCh38"          | "A/C"             |
| chr1:10145    | ["A","G"]  | "GRCh38"          | "A/G"             |
| chr1:10145    | ["A","T"]  | "GRCh38"          | "A/T"             |
| chr1:10147    | ["C","A"]  | "GRCh38"          | "C/A"             |
| chr1:10147    | ["C","G"]  | "GRCh38"          | "C/G"             |
| chr1:10150    | ["C","A"]  | "GRCh38"          | "C/A"             |
| chr1:10150    | ["C","T"]  | "GRCh38"          | "C/T"             |
| chr1:10151    | ["T","A"]  | "GRCh38"          | "T/A"             |
| chr1:10151    | ["T","G"]  | "GRCh38"          | "T/G"             |
| chr1:10152    | ["A","C"]  | "GRCh38"          | "A/C"             |
| chr1:10152    | ["A","G"]  | "GRCh38"          | "A/G"             |
| chr1:10153    | ["A","C"]  | "GRCh38"          | "A/C"             |
| chr1:10155    | ["C","A"]  | "GRCh38"          | "C/A"             |
| chr1:10155    | ["C","G"]  | "GRCh38"          | "C/G"             |
| chr1:10157    | ["T","A"]  | "GRCh38"          | "T/A"             |
| chr1:10157    | ["T","C"]  | "GRCh38"          | "T/C"             |
| chr1:10157    | ["T","G"]  | "GRCh38"          | "T/G"             |
| chr1:10158    | ["A","C"]  | "GRCh38"          | "A/C"             |
| chr1:10158    | ["A","T"]  | "GRCh38"          | "A/T"             |
| chr1:10159    | ["A","C"]  | "GRCh38"          | "A/C"             |
| chr1:10159    | ["A","G"]  | "GRCh38"          | "A/G"             |
| chr1:10163    | ["T","A"]  | "GRCh38"          | "T/A"             |
| chr1:10163    | ["T","C"]  | "GRCh38"          | "T/C"             |
| chr1:10163    | ["T","G"]  | "GRCh38"          | "T/G"             |
| chr1:10164    | ["A","C"]  | "GRCh38"          | "A/C"             |
| chr1:10165    | ["A","C"]  | "GRCh38"          | "A/C"             |
| chr1:10165    | ["A","G"]  | "GRCh38"          | "A/G"             |
| chr1:10169    | ["T","C"]  | "GRCh38"          | "T/C"             |
| chr1:10169    | ["T","G"]  | "GRCh38"          | "T/G"             |
| chr1:10171    | ["A","C"]  | "GRCh38"          | "A/C"             |
| chr1:10171    | ["A","G"]  | "GRCh38"          | "A/G"             |
| chr1:10173    | ["C","G"]  | "GRCh38"          | "C/G"             |
| chr1:10174    | ["C","

In [80]:
mt0.vep.show(10)

+---------------+----------------+-------------------+-------------------+
| locus         | alleles        | vep.assembly_name | vep.allele_string |
+---------------+----------------+-------------------+-------------------+
| locus<GRCh38> | array<str>     | str               | str               |
+---------------+----------------+-------------------+-------------------+
| chr1:10011    | ["C","T"]      | "GRCh38"          | "C/T"             |
| chr1:10020    | ["A","C"]      | "GRCh38"          | "A/C"             |
| chr1:10027    | ["A","G"]      | "GRCh38"          | "A/G"             |
| chr1:10109    | ["AACCCT","A"] | "GRCh38"          | "ACCCT/-"         |
| chr1:10114    | ["T","C"]      | "GRCh38"          | "T/C"             |
| chr1:10114    | ["TA","T"]     | "GRCh38"          | "A/-"             |
| chr1:10119    | ["CT","C"]     | "GRCh38"          | "T/-"             |
| chr1:10120    | ["T","A"]      | "GRCh38"          | "T/A"             |
| chr1:10120    | ["T","C"]      | "GRCh38"          | "T/C"             |
| chr1:10120    | ["TA","T"]     | "GRCh38"          | "A/-"             |
+---------------+----------------+-------------------+-------------------+

+---------------+
| vep.ancestral |
+---------------+
| str           |
+---------------+
| NA            |
| NA            |
| NA            |
| NA            |
| NA            |
| NA            |
| NA            |
| NA            |
| NA            |
| NA            |
+---------------+

+------------------------------------------------------------------------------+
| vep.colocated_variants                                                       |
+------------------------------------------------------------------------------+
| array<struct{aa_allele: str, aa_maf: float64, afr_allele: str, afr_maf: f... |
+------------------------------------------------------------------------------+
| NA                                                                           |
| [(NA,NA,NA,NA,"A/C",NA,NA,NA,10020,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA... |
| [(NA,NA,NA,NA,"A/C/G",NA,NA,NA,10027,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,... |
| [(NA,NA,NA,NA,"ACCCTA/A",NA,NA,NA,10115,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,... |
| [(NA,NA,NA,NA,"T/C",NA,NA,NA,10114,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA... |
| [(NA,NA,NA,NA,"AA/A",NA,NA,NA,10116,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,N... |
| [(NA,NA,NA,NA,"T/-",NA,NA,NA,10120,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA... |
| NA                                                                           |
| [(NA,NA,NA,NA,"T/C",NA,NA,NA,10120,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA... |
| NA                                                                           |
+------------------------------------------------------------------------------+

+-------------+---------+--------+--------------------------------+
| vep.context | vep.end | vep.id | vep.input                      |
+-------------+---------+--------+--------------------------------+
| str         |   int32 | str    | str                            |
+-------------+---------+--------+--------------------------------+
| NA          |   10011 | "."    | "chr1	10011	.	C	T	.	.	GT"      |
| NA          |   10020 | "."    | "chr1	10020	.	A	C	.	.	GT"      |
| NA          |   10027 | "."    | "chr1	10027	.	A	G	.	.	GT"      |
| NA          |   10114 | "."    | "chr1	10109	.	AACCCT	A	.	.	GT" |
| NA          |   10114 | "."    | "chr1	10114	.	T	C	.	.	GT"      |
| NA          |   10115 | "."    | "chr1	10114	.	TA	T	.	.	GT"     |
| NA          |   10120 | "."    | "chr1	10119	.	CT	C	.	.	GT"     |
| NA          |   10120 | "."    | "chr1	10120	.	T	A	.	.	GT"      |
| NA          |   10120 | "."    | "chr1	10120	.	T	C	.	.	GT"      |
| NA          |   10121 | "."    | "chr1	10120	.	TA	T	.	.	GT"     |
+-------------+---------+--------+--------------------------------+

+------------------------------------------------------------------------------+
| vep.intergenic_consequences              

In [62]:
mt0.vep.variant_class.show(10)

,,
locus,alleles,
locus<GRCh38>,array<str>,str
chr1:10011,"[""C"",""T""]","""SNV"""
chr1:10020,"[""A"",""C""]","""SNV"""
chr1:10027,"[""A"",""G""]","""SNV"""
chr1:10109,"[""AACCCT"",""A""]","""deletion"""
chr1:10114,"[""T"",""C""]","""SNV"""
chr1:10114,"[""TA"",""T""]","""deletion"""
chr1:10119,"[""CT"",""C""]","""deletion"""
chr1:10120,"[""T"",""A""]","""SNV"""


In [50]:
mt0.vep.variant_class.summarize()

Non-missing,10000 (100.00%)
Missing,0
Min Size,3
Max Size,9
Mean Size,4.31
Sample Values,"['SNV', 'SNV', 'SNV', 'deletion', 'SNV']"


In [63]:
indels_mt1 = mt0.filter_rows(mt0.vep.variant_class == "insertion")

In [64]:
indels_mt1.count()

(1159, 1061)

In [ ]:
# SNVs : 7,606 (76.1%)
# indels (deletions): 1,235 (12.3%)
# indels (insertions): 1,159 (11.6%)

In [18]:
mt0.variant_type.show()

,,
locus,alleles,variant_type
locus<GRCh38>,array<str>,str
chr1:10011,"[""C"",""T""]",NA
chr1:10020,"[""A"",""C""]",NA
chr1:10027,"[""A"",""G""]",NA
chr1:10109,"[""AACCCT"",""A""]","""indel"""
chr1:10114,"[""T"",""C""]",NA
chr1:10114,"[""TA"",""T""]",NA
chr1:10119,"[""CT"",""C""]",NA
chr1:10120,"[""T"",""A""]",NA


In [24]:
mt0.variant_type.summarize()

Non-missing,4704 (47.04%)
Missing,5296 (52.96%)
Min Size,3
Max Size,11
Mean Size,4.04
Sample Values,"['indel', 'mixed', 'mixed', 'multi-indel', 'multi-indel']"


In [19]:
indels_mt = mt0.filter_rows(mt0.variant_type == "indel")

In [26]:
indels_mt.rows().select().show(5)

,
locus,alleles
locus<GRCh38>,array<str>
chr1:10109,"[""AACCCT"",""A""]"
chr1:10166,"[""CCCTAACCCTAACCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAAACCCTAAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCAACCCCAACCCCAACCCCAACCCCAACCCCAACCCTAACCCCTAACCCTAACCCTAACCCTACCCTAACCCTAACCCTAACCCTAACCCTAA"",""C""]"
chr1:10168,"[""CTAACCCTAACCT"",""C""]"
chr1:10172,"[""CCCTAA"",""C""]"
chr1:10616,"[""CCGCCGTTGCAAAGGCGCGCCG"",""C""]"


In [20]:
indels_mt.count()

(287, 1061)